In [30]:
import pandas as pd
import torch
from transformers import AdamW,    get_linear_schedule_with_warmup, AutoTokenizer, GPT2Config, GPT2LMHeadModel
from torch.utils.data import Dataset, random_split, DataLoader, RandomSampler, SequentialSampler
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
poem_df = pd.read_csv('poems.csv')
poem_df = poem_df.fillna('')
random_seed = 1
batch_size= 2
epochs=8
max_length = 100
data_df = poem_df.stanza_text

In [49]:
model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
special_tokens_dict = {
    'bos_token': '<BOS>', 
    'eos_token': '<EOS>', 
    'pad_token': '<PAD>',
    'cls_token': '<CLS>',
    'sep_token': '<SEP>',
    }
num_added_tokens = tokenizer.add_special_tokens(special_tokens_dict)
num_added_tokens

5

In [50]:
from dataset import PoemDataset
poem_dataset = PoemDataset(
    data_df,
    tokenizer=tokenizer,
    max_length=100
)


In [51]:
from utils import train_val_split
train_set, val_set = train_val_split(split=0.8, dataset=poem_dataset)

In [52]:
torch.cuda.manual_seed_all(random_seed)

In [53]:
train_loader = DataLoader(
    dataset=train_set,
    sampler=RandomSampler(train_set),
    batch_size=batch_size
)
val_loader = DataLoader(
    dataset=val_set,
    sampler=SequentialSampler(val_set),
    batch_size=batch_size
)

In [54]:
learning_rate = 1e-4
eps = 1e-8
warmup_steps = 50
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

promt = "<BOS>"
generated = torch.tensor(tokenizer.encode(promt)).unsqueeze(0)
generated = generated.to(device)

In [55]:
configuration = GPT2Config(vocab_size=len(tokenizer), n_positions=max_length).from_pretrained('gpt2', output_hidden_states=True)

model = GPT2LMHeadModel.from_pretrained(model_name, config=configuration)
model.resize_token_embeddings(len(tokenizer))

Embedding(50262, 768)

In [56]:
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=learning_rate, eps=eps)
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps = warmup_steps,
    num_training_steps = total_steps
)

In [57]:
import numpy as np
from tqdm import tqdm
epochs = 100
for epoch in tqdm(range(epochs)):
    print(f'epoch {epoch+1} of {epochs}')
    epoch_loss = list()
    model.train()
    for step, batch in enumerate(train_loader):
        input_ids, attn_masks = batch
        input_ids = input_ids.to(device)
        attn_masks = attn_masks.to(device)

        outputs = model(
            input_ids,
            labels=input_ids,
            attention_mask=attn_masks,
        )
        optimizer.zero_grad()

        loss = outputs[0]
        loss.backward()
        optimizer.step()
        scheduler.step()
        epoch_loss.append(epoch_loss)

    avg_loss = np.mean(epoch_loss)
    print(f'loss {avg_loss}')

  0%|          | 0/100 [00:00<?, ?it/s]


epoch 1 of 100


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 8.00 GiB total capacity; 6.62 GiB already allocated; 0 bytes free; 6.65 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF